In [1]:
import json
import pandas as pd
import numpy as np
import dill as pickle
import scipy.sparse
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.snowball import EnglishStemmer
from nltk.stem import WordNetLemmatizer
from functools import lru_cache
from tqdm import tqdm as tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import os,sys,time

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')
        elif(format == 'npz'):
            data = scipy.sparse.load_npz(file)

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')
        elif(format == 'npz'):
            scipy.sparse.save_npz(file, data)
        return

In [2]:
# load data
iformat = 'csv'
oformat = 'hdf'
DataBase = '../data'
DataSet = {}
for mod in ['train', 'test']:
    DataSet[mod] = DataUtil2.load('%s/raw/%s.%s' % (DataBase, mod, iformat), iformat)
    DataSet[mod]['comment_text'] = DataSet[mod]['comment_text'].fillna('nan')
print('load data done.')

load data done.


In [3]:
stemmer = EnglishStemmer()

@lru_cache(30000)
def stem_word(text):
    return stemmer.stem(text)

lemmatizer = WordNetLemmatizer()

@lru_cache(30000)
def lemmatize_word(text):
    return lemmatizer.lemmatize(text)

def reduce_text(conversion, text):
    return " ".join(map(conversion, wordpunct_tokenize(text.lower())))

def reduce_texts(conversion, texts):
    return [reduce_text(conversion, str(text))
            for text in tqdm(texts)]

for mod in ['train', 'test']:
    DataSet[mod]['comment_text_stemmed'] = reduce_texts(stem_word, DataSet[mod]['comment_text'])
    #DataSet[mod]['comment_text_lemmatized'] = reduce_texts(lemmatize_word, DataSet[mod]['comment_text'])

100%|██████████| 226998/226998 [00:13<00:00, 16736.99it/s]


In [10]:
label2binary = np.array([
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 1],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 1],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 1, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 1],
    [0, 0, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 1, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 1, 1],
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 1],
    [0, 1, 1, 0, 1, 0],
    [0, 1, 1, 0, 1, 1],
    [0, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 0, 1],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 1, 1],
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 1, 0, 1],
    [1, 0, 0, 1, 1, 0],
    [1, 0, 0, 1, 1, 1],
    [1, 0, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 0],
    [1, 0, 1, 0, 1, 1],
    [1, 0, 1, 1, 0, 0],
    [1, 0, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 0],
    [1, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1],
    [1, 1, 0, 0, 1, 0],
    [1, 1, 0, 0, 1, 1],
    [1, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 0, 1],
    [1, 1, 0, 1, 1, 0],
    [1, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 0],
    [1, 1, 1, 0, 0, 1],
    [1, 1, 1, 0, 1, 0],
    [1, 1, 1, 0, 1, 1],
    [1, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1],
])

OutputDir = '../data/meta/kfold/'
strategy = 'lr_word_char_tfidf'
targets = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def metric(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    columns = y_true.shape[1]
    column_losses = []
    for i in range(0, columns):
        column_losses.append(log_loss(y_true[:, i], y_pred[:, i]))
    return np.array(column_losses).mean()

def cv(model, X, y, label2binary, n_splits=3):
    def split(X, y):
        return StratifiedKFold(n_splits=n_splits).split(X, y)
    
    def convert_y(y):
        new_y = np.zeros([len(y)])
        for i, val in enumerate(label2binary):
            idx = (y == val).max(axis=1)
            new_y[idx] = i
        return new_y
    
    X = np.array(X)
    y = np.array(y)
    scores = []
    fold = 0
    for train, test in tqdm(split(X, convert_y(y)), total=n_splits):
        FoldOutput = '%s/%s' % (OutputDir, fold)
        if(os.path.exists(FoldOutput) == False):
            os.makedirs(FoldOutput)
        fitted_model = model(X[train], y[train])
        predict = fitted_model(X[test])
        score = metric(y[test], predict)
        scores.append(score)
        FoldOutputFile = '%s/valid_%s.csv' % (FoldOutput, strategy)
        df = pd.DataFrame(index= range(len(predict)))
        #df.columns = ['%s_%s' % (strategy,t) for t in targets]
        for idx in range(len(targets)):
            df['%s_%s' % (strategy, targets[idx])] = predict[:,idx]
        DataUtil2.save(df, FoldOutputFile, 'csv', 6)
        print('fold %s, cv score %.4f' % (fold, score))
        fold += 1
    return np.array(scores)

In [11]:
import sys,os

# other_feats = ['total_length', 'capitals', 'caps_vs_length', 'num_exclamation_marks', 'num_question_marks', 'num_punctuation', 'imcomplete_punctuation', 'question_mask_ratio', 'exclamation_mark_ratio', 
#                'imcomplete_punctuation_ratio', 'num_words', 'num_unique_words', 'words_vs_unique', 'num_smilies', 'similes_ratio']

def regression_wordchars(X, y):
    tfidf_word = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='word',
        min_df=3, 
        max_df=0.9,
        use_idf= 1,
        smooth_idf= 1,
        ngram_range=(1,1),
        max_features=20000
    )
    X_tfidf_word = tfidf_word.fit_transform(X[:, 1])
    tfidf_char = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='char', 
        ngram_range=(1, 4),
        max_features=20000,
        lowercase=False)
    X_tfidf_char = tfidf_char.fit_transform(X[:, 0])
    ##
    feats = [X_tfidf_word, X_tfidf_char]
#     for i in range(len(other_feats)):
#         feats.append(sparse.csr_matrix(np.reshape(X[:, 2+i], (X[:, 2+i].shape[0], 1)).tolist()))
#         print('convert for %s done...' % other_feats[i])
    #
    X_tfidf = sparse.hstack(feats)
    print('feature reconstruction pass...')
    
    columns = y.shape[1]
    regressions = [
        LogisticRegression(C= 4).fit(X_tfidf, y[:, i])
        for i in range(columns)
    ]
    
    def _predict(X):
        X_tfidf_word = tfidf_word.transform(X[:, 1])
        X_tfidf_char = tfidf_char.transform(X[:, 0])
        feats = [X_tfidf_word, X_tfidf_char]
#         for i in range(len(other_feats)):
#             feats.append(sparse.csr_matrix(np.reshape(X[:, 2+i], (X[:, 2+i].shape[0], 1)).tolist()))
#             print('convert for %s done...' % other_feats[i])
        X_tfidf = sparse.hstack(feats)
        predictions = np.zeros([len(X), columns])
        for i, regression in enumerate(regressions):
            regression_prediction = regression.predict_proba(X_tfidf)
            predictions[:, i] = regression_prediction[:, regression.classes_ == 1][:, 0]
        return predictions
    
    return _predict

cols = ['comment_text', 'comment_text_stemmed']
# DataSet['train'][other_feats] = DataSet['train'][other_feats].astype(float)
# cols.extend(other_feats)
ret = cv(regression_wordchars,
   DataSet['train'][cols].values,
   DataSet['train'][['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
   label2binary)
print(ret)

  0%|          | 0/3 [00:00<?, ?it/s]

feature reconstruction pass...


 33%|███▎      | 1/3 [01:53<03:46, 113.48s/it]

fold 0, cv score 0.0475
feature reconstruction pass...


 67%|██████▋   | 2/3 [03:48<01:54, 114.14s/it]

fold 1, cv score 0.0483
feature reconstruction pass...


100%|██████████| 3/3 [05:44<00:00, 114.80s/it]

fold 2, cv score 0.0488
[ 0.04746066  0.0483406   0.04879103]


In [12]:
%%time
model = regression_wordchars(np.array(DataSet['train'][['comment_text', 'comment_text_stemmed']]),
                             np.array(DataSet['train'][['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]))

feature reconstruction pass...
CPU times: user 2min 32s, sys: 668 ms, total: 2min 33s
Wall time: 2min 33s


In [13]:
%%time
prediction = model(np.array(DataSet['test'][['comment_text', 'comment_text_stemmed']]))

CPU times: user 2min 51s, sys: 11.8 s, total: 3min 3s
Wall time: 3min 3s


In [14]:
submission = pd.DataFrame()
submission['id'] = DataSet['test']['id']
for i, label in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    submission[label] = prediction[:, i]
# print(submission.tail(100))

In [15]:
import sys,os,datetime

SubmitOutputDir = '../data/meta/submit'
if(os.path.exists(SubmitOutputDir) == False):
    os.makedirs(SubmitOutputDir)
SubmitFileName = '%s_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
submission.to_csv('%s/%s.csv' % (SubmitOutputDir, SubmitFileName), index= None)
print('zip %s/%s.zip %s/%s.csv' % (SubmitOutputDir, SubmitFileName, SubmitOutputDir, SubmitFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitOutputDir, SubmitFileName, SubmitOutputDir, SubmitFileName))

zip ../data/meta/submit/lr_word_char_tfidf_2018-01-18.zip ../data/meta/submit/lr_word_char_tfidf_2018-01-18.csv


0